Baseline Model

# CNR Parking EDA Notebook

This data will be prepared in preperation for training into a model

In [ ]:
#imports
import os
import cv2
import torch
import random
import zipfile
import numpy as np
import torch.nn as nn
import pandas as pd
import seaborn as sns
from PIL import Image
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

# Datasets

CNRPark+EXT.csv (18.1 MB)

CSV collecting metadata for each patch of both CNRPark and CNR-EXT datasets

CNRPark-Patches-150x150.zip (36.6 MB)

segmented images (patches) of parking spaces belonging to the CNRPark preliminary subset.
Files follow this organization: <CAMERA>/<CLASS>/YYYYMMDD_HHMM_<SLOT_ID>.jpg, where:

<CAMERA> can be A or B,
<CLASS> can be free or busy,
YYYYMMDD_HHMM is the zero-padded 24-hour capture datetime,
<SLOT_ID> is a local ID given to the slot for that particular camera

In [ ]:
# CSV file of both park_ext data
park_ext = "https://github.com/fabiocarrara/deep-parking/releases/download/archive/CNRPark+EXT.csv"

# segmented images beloning to CNR_Park subset
cnr_park = "https://github.com/fabiocarrara/deep-parking/releases/download/archive/CNR-EXT-Patches-150x150.zip"

# segmented images of for CNR_EXT subset
cnr_ext = "https://github.com/fabiocarrara/deep-parking/releases/download/archive/CNR-EXT-Patches-150x150.zip"

df = pd.read_csv(park_ext)

df.head()

<ipython-input-43-332b11c17138>:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(park_ext)


,camera,datetime,day,hour,image_url,minute,month,occupancy,slot_id,weather,year,occupant_changed
0,A,20150703_0805,3,8,CNRPark/A/free/20150703_0805_1.jpg,5,7,0,1,S,2015,NaN
1,A,20150703_0810,3,8,CNRPark/A/free/20150703_0810_1.jpg,10,7,0,1,S,2015,NaN
2,A,20150703_0815,3,8,CNRPark/A/busy/20150703_0815_1.jpg,15,7,1,1,S,2015,0.0
3,A,20150703_0820,3,8,CNRPark/A/busy/20150703_0820_1.jpg,20,7,1,1,S,2015,0.0
4,A,20150703_0825,3,8,CNRPark/A/busy/20150703_0825_1.jpg,25,7,1,1,S,2015,0.0


In [ ]:
# Display info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157549 entries, 0 to 157548
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   camera            157549 non-null  object 
 1   datetime          157549 non-null  object 
 2   day               157549 non-null  int64  
 3   hour              157549 non-null  int64  
 4   image_url         157549 non-null  object 
 5   minute            157549 non-null  int64  
 6   month             157549 non-null  int64  
 7   occupancy         157549 non-null  int64  
 8   slot_id           157549 non-null  int64  
 9   weather           157549 non-null  object 
 10  year              157549 non-null  int64  
 11  occupant_changed  87618 non-null   float64
dtypes: float64(1), int64(7), object(4)
memory usage: 14.4+ MB


In [ ]:

# paths
zip_path = "/content/CNR-EXT-Patches-150x150.zip"
extract_path = "/content/CNR-EXT-Patches"

# Download and extract images
print("Downloading dataset...")
urlretrieve("https://github.com/fabiocarrara/deep-parking/releases/download/archive/CNR-EXT-Patches-150x150.zip", zip_path)

print("Extracting dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    zip_contents = zip_ref.namelist()

#print("Dataset extracted successfully!")
#for file in zip_contents:
    #print(file)


Extracting dataset...


In [ ]:
#Name list of files extracted
extracted_files = os.listdir(extract_path)
print(extracted_files)

['LABELS', 'PATCHES']


In [ ]:
!ls /content


CNR-EXT-Patches		     model_architecture.png    uner_forward_propagation.pdf
CNR-EXT-Patches-150x150.zip  sample_data	       unet_forward_propagation
model_architecture	     uner_forward_propagation  unet_forward_propagation.png


In [ ]:
class ParkingDataset(Dataset):
    def __init__(self, patch_dir, label_dir, transform=None):
        self.patch_dir = patch_dir
        self.label_dir = label_dir
        self.transform = transform
        self.patches = []
        self.labels = {}

        # Read label files
        for label_file in os.listdir(label_dir):
            if label_file.endswith(".txt"):
                camera_id = label_file.split(".txt")[0]  # remove .txt
                label_path = os.path.join(label_dir, label_file)

                with open(label_path, "r") as file:
                    for line in file:
                        parts = line.strip().split(" ")  # split
                        if len(parts) == 2:
                            img_relative_path, label = parts
                            full_img_path = os.path.join(patch_dir, "PATCHES", img_relative_path)  # full path
                            self.labels[full_img_path] = int(label)

        # image paths
        for root, _, files in os.walk(patch_dir):
            for file in files:
                full_path = os.path.join(root, file)
                if full_path in self.labels:  # images match full path
                    self.patches.append(full_path)

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
     patch_path = self.patches[idx]
     label = self.labels.get(patch_path, 0)  # Default to 0 if missing

     patch = Image.open(patch_path).convert("RGB")

     if self.transform:
        patch = self.transform(patch)

    # 128x128 tensor
     label = torch.full((128, 128), label, dtype=torch.long)

     return patch, label


#set paths
patch_dir = "/content/CNR-EXT-Patches"
label_dir = "/content/CNR-EXT-Patches/LABELS"


In [ ]:
#Data Aumentation
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2,saturation=0.2),
    transforms.ToTensor()
])

In [ ]:
train_data = ParkingDataset(patch_dir,label_dir,transform)
train_load = DataLoader(train_data, batch_size=8, shuffle= True)


In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
import torch.optim as optim
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Define the RCNN model
class RCNN(nn.Module):
    def __init__(self, num_classes):
        super(RCNN, self).__init__()

        # Backbone network - Here we use ResNet as a backbone
        self.backbone = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])  # Remove the final fully connected layers

        # Region Proposal Network (RPN)
        self.rpn_conv1 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.rpn_cls = nn.Conv2d(64, 32, kernel_size=1)  # Objectness scores for 9 proposals
        self.rpn_reg = nn.Conv2d(64, 30, kernel_size=1)  # Bounding box regression (4 coordinates for each of the 9 proposals)

        # RoI Pooling (simplified)
        self.roi_pool = torchvision.ops.MultiScaleRoIAlign(
            output_size=(7, 7), spatial_scale=1.0/16, sampling_ratio=2
        )

        # Classifier and Bounding Box Regressor
        self.classifier_fc = nn.Linear(128*7*7, 2048)
        self.classifier_out = nn.Linear(2048, num_classes)
        self.bbox_fc = nn.Linear(2048, 4)

    def forward(self, x):
        # Extract feature map using backbone
        features = self.backbone(x)

        # Apply RPN to generate objectness scores and bounding box regressions
        rpn_features = F.relu(self.rpn_conv1(features))
        objectness = self.rpn_cls(rpn_features)
        bbox_preds = self.rpn_reg(rpn_features)

        # Generate region proposals (you would typically use a proposal generator here)
        # For simplicity, we assume fixed proposals (this step is simplified in this case)
        proposals = self.generate_proposals(objectness, bbox_preds)

        # RoI Pooling on the features
        pooled_features = self.roi_pool(features, proposals)

        # Flatten pooled features
        pooled_features = pooled_features.view(pooled_features.size(0), -1)

        # Pass through classifier and regressor
        x_class = self.classifier_fc(pooled_features)
        x_class = F.relu(x_class)
        class_scores = self.classifier_out(x_class)

        bbox_preds = self.bbox_fc(x_class)

        return class_scores, bbox_preds

    def generate_proposals(self, objectness, bbox_preds):
        return objectness


In [ ]:
# Initialize and train the model
def train_rcnn(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, annotations in train_loader:
            optimizer.zero_grad()

            # Forward pass
            class_scores, bbox_preds = model(images)

            # Calculate loss
            loss = criterion(class_scores, bbox_preds, annotations)  # You would define this custom loss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")

In [ ]:
train_loader = DataLoader(CNRparkDataset(image_paths=[''], annotations=[]), batch_size=4, shuffle=True)
model = RCNN(num_classes=2)  # Assuming binary classification for car/no car
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # You need a custom loss function combining classification and bbox regression losses
train_rcnn(model, train_loader, optimizer, criterion)

In [ ]:
#grid search to find best learning rate, batch size,
#may be try resnet 101 instead of resnet 50
#change all images to grayscale to see what that does